<img src="https://drive.google.com/uc?export=view&id=1FUSWDRDDflxrUrYBFPKHE50KrelBYO6N" style="width:50%" align="centre">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/University_of_Sussex_Logo.svg/1024px-University_of_Sussex_Logo.svg.png" style="width:50%;"  height="500" width="500" align="centre">



<!---weird that to put it in the middle I have to align it to the right-->

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dark matter: Searching for the invisible
#Workbook 3: Using machine learning



## About ##

Hello and welcome to the second part on finding dark matter from the ATLAS experiment. The ultimate goal of this part is to create a tool to optimise the inputs of event features to automatically determine the signal from the background, as you did manually in Workbook 2. We will create a type of machine learning model called a binary classifier. This will be trained to tell the difference between signal and background (thus being binary). Signal will be labelled with a **1** and background with a **0**.

<br>
HOW TO USE NOTEBOOK:

* Run each code cell one at a time, if you leave the page replay every cell.
* Fill in missing code in "Do it yourself" sections. 
* It is divided into sections that take around TIME to complete. 
* I recommend taking a break between each section to collect your thoughts. 
* There is a quiz at the end of each section to test your knowledge.



## 1: Intro##

Experimental particle physics involves a lot of data analysis. The LHC produces up to 1 billion proton-proton collisions per second. This results in a tremendous amount of data (around one petabyte per day, or $10^{6}$ gigabytes) which must be understood to find the underlying physics. Machine learning really benefits from this large amount of data allowing for effective models to be trained. The efficiency and speed of machine learning methods also results in a massive decrease in computing time compared with manual methods (such as the cutting done in Workbook 2).
At CERN, machine learning is used in many ways: for triggering, for event selection (what we shall be doing) and for particle and jet identification. See more on other uses [here](https://sparks.cern/ai-cern).



This workbook will be using the python module scikit-learn to create the classifiers. This machine learning module is robust enough for us to make full use of machine learning models but is still readable and simple to initialise. If you wish to learn about more advanced machine learning libraries, I recommend PyTorch, TensorFlow, and especially Keras (all python modules).

We will also need NumPy and pandas so we shall import these modules:


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import ipywidgets as widgets #used to create interactive elements within the notebook

plt.style.use('seaborn-deep') # plot styling

We will also set a seed for this project, so the results are reproduceable:

In [ ]:
from numpy.random import seed
#set seed value for all code
seed_value = 420
seed(seed_value)

## 2: What is machine learning? ##

Machine learning (ML) is a type of **artificial intelligence**, which is the computing field of developing computer programs/algorithms that perform tasks that typically require human intelligence, such as problem-solving. This workbook will focus on supervised learning. This is when we train a machine learning model with example data of what we are looking for.

**Example:** Let's say we want to teach a computer to recognise a dog in a picture. We will train the computer on lots of images of animals while telling the program what images correspond to dogs. This will teach the computer to correctly recognise images of dogs.


### 2.1: General principles ###

Machine learning models are computer algorithms that are trained to achieve specific goals without being programmed with explicit instructions on how to achieve them; they are not *programmed* but *trained*.

There are three main tasks of machine learning:
1.   **Regression:** The input is a set of multi-feature data points, and the output is a real number or a sequence of real numbers (e.g. inputs could be the features of a house and the output could be a predicted house price, [link](https://medium.com/codex/house-price-prediction-with-machine-learning-in-python-cf9df744f7ff)). 
2.   **Classification:** The input is a set of multi-feature data points and the output is an integer which represents different classes (e.g. based on what is contained within an email, a machine learning model could sort between mail and spam, [link](https://towardsdatascience.com/email-spam-detection-1-2-b0e06a5c0472)).
3.   **Generation:** The input is noise and the output is something desirable (E.g. Input is a random seed and the output is generated images of people that don’t exist, [link]( https://www.boredpanda.com/ai-image-generation-fake-faces-people-nvidia/?utm_source=google&utm_medium=organic&utm_campaign=organic)).


In this notebook we shall be training a **classifier**.
*   We will have <font color='#819468'>**objects**</font> which we are interested in.

*   These objects need to be <font color='#688194'>**classified**</font> into a set number of groups.

*   To do this the <font color='#946881'>**features**</font> of the objects are used.



#### 2.1.1: Pre-processing and notation ####

Before training our machine learning models we must prepare the data so the computer can make sense of it. We define a data set called $X$ which holds all the objects (with $x$ as a single object) and a $y$ data set which will hold all the labels that correspond to the objects (the $X$ data set).

As well as doing this we will split the data up into two parts: training data and testing data. The purpose of these data sets is evident in their names. The classifier is trained on the training data then tested on the testing data - this is discussed further in Section 3.1 but the key point is that the testing data is new unseen data for the classifier to be tested on after training.


#### 2.1.2: How does classification work? ####

Lets take a step back from particle physics and consider different objects, lets say spherical fruit (e.g. apple, orange, tomato, etc.). We can identify the type of fruit using two features: colour and average radius. So we define an object as

$x\left (c, r  \right )$.

We want to create an algorithm that can classify these <font color='#819468'>objects</font> into two groups: lemons and everything else (e.g. grapes, oranges, apples, etc.). Lets say we have two <font color='#819468'>objects</font>:
 
$x_{1}\left (yellow, 2cm  \right ) = \text{a lemon}$, 

$x_{2}\left (green, 8cm  \right ) =  \text{NOT a lemon (a watermelon)}$.

We can plot these two points in a 2D graph with colour and radius as the dimensions:
![MLd1_f](https://lh4.googleusercontent.com/-6YTfe0Nt1YOBBCb3-AreVNNjjyQeiSOlO0FZv_d8fPJC-VblzWN017DgcseX_DY8zE=w2400)





These points are the training data. The algorithm is explicitly told that these are, or are not, lemons, soo it is using them to learn how to identify a lemon. Let's add more <font color='#819468'>objects</font> to train the algorithm:

![MLd2_f](https://lh5.googleusercontent.com/bMdPJpENrYD1Zgs3zxv_7HtloOMb93eM8bS23THq-8jSFs655MEf_9qGoEF1UT9WXR0=w2400)

These <font color='#819468'>objects</font> make up the training data, The machine learning model uses them to 
find the boundaries of this plot to <font color='#688194'>classify</font> the two groups:

![MLd3_f](https://lh3.googleusercontent.com/7YDZ5amhf3SvfRafzZN8GvKAKIkGTVF_9rHhOJRKJLjmsXB4PzikgC9_rReG2LpNtDc=w2400)

Then from these boundaries, the model can predict if a new <font color='#819468'>object</font> is a lemon or not a lemon by looking at where it is on the plot. So, let's introduce a new fruit that has not been seen. This is considered the testing data as it is unlabelled and was not seen by the machine learning model during training:

![MLd4_f](https://lh5.googleusercontent.com/C-H8k2ctNTziBRKi4I3sUZbVJRbXwB3fE2bbHZWiNc_Fq2UDxDjGUquk5yMB35BLiuU=w2400)

The machine learning algorithm doesn’t know what it is 
supposed to be, and has to use what it has learnt from the training data to 
guess if it is an orange or not. In this case, it would guess that the new fruit is not a lemon.

Obviously these two features are not enough to identify every type of fruit correctly. So, let's introduce more! Let's say we now have $N$ <font color='#946881'>features</font> (new features such as volume or colour brightness). Now our graph will be in $N$ dimensions. This is not plottable but the same logic as for 2-D is upheld:
the machine learning model will find the part of this $N$-dimensional space (hyperspace) which belongs to lemons. It recognises complex pattens in the data.

This is how machine learning models are trained to classify objects.


#### 2.1.3: What is training? ####

We have been talking about training a lot, but what does it entail? In machine learning, we teach computer algorithms and test them to see how good they are at doing their job. We then give feedback to the algorithm which it uses to become more accurate. This process iterates until the algorithm can do its task to a set standard. Over time, the algorithm changes its logic to produce accurate results.


### 2.2: Our application ###

So now our $N$ dimensions are the features of the collision events (the same properties we were cutting in Workbook 2). We will train our classifier to recognise the complex patterns in the data to identify the signal events from the background ones.

We have objects:
<font color='#819468'>
*   **Collision events** from the Monte Carlo simulation (**1.2 million events**)
</font>

With the goal:
<font color='#688194'>
*   Classify **signal** from **background** (labelled as **1 or 0**)</font>

To do this, we will use the features of the objects (**N = 7**):
<font color='#946881'>
*   **Leading lepton's transverse momentum** 
*   **Sub-leading lepton's transverse momentum** 
*   **Dilepton invariant mass** ($m_{ll}$)
*   **Missing transverse energy** ($E_{Tmiss}$)
*   **Angle between $E_{Tmiss}$ and net momentum of the dilepton** ($\Delta \phi$)
*   **Fractional transverse momentum difference** 
*   **Missing transverse energy over total momentum** $E_{Tmiss}/H_T$
</font>


To get the machine learning model to recognize the signal from background we shall train it on all the processes:

<table>
<thead>
<tr><th>Non-resonant lepton pair production</th><th>Z+ boson jets</th><th>W and Z boson production</th><th>Double Z boson production</th><th>Dark matter production</th></tr>
</thead>
</table>

In [ ]:
#Lets made an array to hold the names of the processes
processes = ['nonres',
              'Zjets',
              'WZ',
              'ZZ',
              'DM_300']

As before we consider every process except dark matter production (DM 300) to be the background. Now let's get load the data we need to do the analysis:

In [ ]:
#Option to use local file if you download the notebook locally, will run better with slower internet speeds
files_local = True

#This dictionary will hold all the event data for each process 
All_processes = {} 

#We now loop through all the processes and read the data from either local files or from the web address
index = 0  #This index is for identifying the process, will increment while looping

for process in processes: 
    if files_local:
        All_processes[process] = pd.read_csv('/content/drive/MyDrive/Data/'+process+'.csv')
    else:
        All_processes[process] = pd.read_csv('https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/csv/DM_ML_notebook/'+process+'.csv') # read data files
    
    #We will add a process ID to the data set to help with identification
    All_processes[process]["processID"] = index
    index = index + 1

FileNotFoundError: ignored

Now that we have loaded the data let's go over the two types of classifiers we will be training.

### 2.3: Random forest ###

The first method we shall be going over is the **random forest**. This is a black box, ensemble learning method using **decision trees**. Now, there is a lot to unpack in that sentence but that is the technical description. This method works by creating a set number of [decision trees](https://www.youtube.com/watch?v=_L39rN6gz7Y) of a set depth - these are themselves machine learning models - then using them all in unison to make classifications.

Decision trees are a structure of repeated binary cuts on individual variables (exactly like you did manually in Workbook 2) in a flowchart like layout. Cuts are made at decision nodes. Leaf nodes are where the cuts end and a classification is made. When this happens is determined by a criterion that measures the quality of a cut. Examples include gini, entropy and log loss (read more about this in Section 2.3.1). The diagram below shows such a tree:


![Dtree](https://lh3.googleusercontent.com/93Lh9CaVLKpS2EYyOa-a5T5FZpRR2uMWqGiHeST_zbk3jA_JKynhxEi7eFhziW_9kOY=w2400)


The model is optimised at each node to give the best separation of signal and background. They are very susceptible to overtraining (more on this later), but random forests overcome this.

Each tree is trained using a random sample of $X$ with replacement (this is called **bootstrapping**) and a random selection of features to use for each tree (each tree can specialise). Each tree is then trained independently. Any new data point is passed through each tree to get different predictions. These predictions are combined to get a final one, this is called **aggregation**, and follows the same logic as "wisdom of the crowd".

The diagram below shows the layout of a random forest made up of decision trees:
![RF_DIA](https://lh3.googleusercontent.com/FzS39r5yR5K0daoJVKSj7eztT4btXsg2Owp5LRaXHrsPBxi_Qmhl_1adoINzrVtan9Y=w2400)


Random forests have several **advantages**:
* Overfitting is reduced due to ensemble learning, improving their accuracy;
* Works well with both discrete and continuous variables;
* Normally uneffected by outliers and missing values, which they can handle automatically;
* The algorithm is very stable;
* It is less affected by noise;
* The feature importance can be identified.

Yet, they also come with some **disadvantages**:
* They require a large amount computational power resulting in longer training periods;
* They have added complexity when lots of decision trees are trained.



#### 2.3.1: Advanced section: Gini, entropy and log loss

In a nutshell, each of the three functions measure the quality of a cut in a decision tree, but in distinctly different ways. However, the exact mathematical formulations for each of them can be quite involved, even for regular users of machine learning. In the following, the interested reader may find rough descriptions, but you are encouraged to do any further reading yourself if you want more information.

* **Gini impurity** measures the probability that data is mislabelled by the node as the incorrect class. If this probability is **0**, the node is pure (all data contained at the node are of the same class) and the decision tree will no longer split at that node. Under this method, the aim is for splits to be made which minimise gini impurity of nodes. More information may also be found [here](https://en.wikipedia.org/wiki/Decision_tree_learning). 

* **Entropy** calculates the disorder of features at a node. Similarly to gini impurity, the aim is for this to be minimised when a feature is used to split the data. Although it seems that gini and entropy are basically the same, more information can be found [here](https://quantdare.com/decision-trees-gini-vs-entropy/) which further outlines the subtle differences.

* **Log loss** is a metric which is also sought to be minimised. The log loss is usually $-1 \times log(L)$ where $L$ is something called a likelihood function. Liklihood functions provide a way to assess how well an observed outcome matches an expected one. In the context of decision trees, this is an assessment of how well the binary classification of a node corresponds to the actual makeup of data at that node. Further information can be found [here](https://towardsdatascience.com/intuition-behind-log-loss-score-4e0c9979680a) and [here](https://www.kaggle.com/code/dansbecker/what-is-log-loss/notebook).

Further information on the three methods can also be found [here](https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation) and by a look at the variety of sources on the web. Yet, again, none of this information is needed to complete this workbook!!!

### 2.4: Neural network ###

The second method is the **neural network**, a mathematical function inspired by the biological neural networks of animal brains. It is made up of nodes/artificial neurons connected by weights. These weights signify the importance of each connection to make the correct prediction. Each neuron acts as a function by having a value dependent on those weights. 

 Neural networks are constructed of layers of nodes:

*   Input layer
*   Hidden layers
*   Output layer








The input layer has a node for each dimension of the input. Here, these are the event features. Hidden layers exists which can be a varying size (the size is set before training and is called a hyperparameter) and there is an output layer made up of nodes which correspond to each classification group. 

The values of the input nodes correspond to the features of the objects to be identified and the output node vaules will correspond to what the object will be classified as.

The diagram below shows the layout of a nodes and weights that make up a neural network:
![NN_DIA](https://lh5.googleusercontent.com/L-bJ7gYuPDcO6lMqKg1LXx4inag-SR8afwMqnyfOZSKCKn3NBeUY7dEvS9MGar_hHnc=w2400)

Again, as well as the advantages of machine learning as whole, neural networks have several other **advantages**:
* Continuous Learning: a neural network can be trained continuously to improve its performance over time (it does not need to be retrained when training on more data);
* They have a parallel processing capability;
* There is an error tolerance due to the complexity of neurons;
* Neural netweorks store the information of the data they are trained on
* They are flexible.

Yet, they also comes with some **disadvantages**:
* Due to their parallel processing, they can be hardware dependant;
* The algorithms can become very complex when used for specified tasks; 
* They have a black box nature: this means that the logic in which the neural network makes decisions is not understandable by humans due to its complexity;
* The results are only approximations;
* They are very dependent on the data: the whole nature of a neural network is dependent on the data it is trained on so erroneous data will, in turn, affect it. 


Try experimenting with different layer sizes to see how the neural network is constructed. Run the hidden cell below this, then in the next cell set what parameters you want.

In [ ]:
#@title
def NeuralNetworkPlot(input_node_size,hidden_layer_sizes,output_node_size):
    
    fig = plt.gcf()
    ax = fig.gca()
    
    scale = 0.2
    
    layer_widths = [input_node_size]
    
    for y in range(0,hidden_layer_sizes[1]):
        layer_widths.append(hidden_layer_sizes[0])
    layer_widths.append(output_node_size)
    
    max_width = max(layer_widths)
    
    buffer =  ( max_width - np.array(layer_widths) )/2
    #print(buffer)
    node_locations = []
    
    for i,o in zip(layer_widths,buffer):
        node_locations.append(np.linspace(0, i-1,i)+o)
    
    to_be_plotted = 0
    
    
    
    for i in range(0,len(node_locations)):
        layer = node_locations[i]
        if i == len(node_locations) -1:
            break
        next_layer = node_locations[i+1]
        for o in range(0,len(layer)):
            
            
            
            
            
            node = layer[o]
            #print(node)
            #print("=======",i,i+1)
            original = np.full(len(next_layer),node)
            
            #print(original) 
            
            next_ = next_layer
            #print(next_layer)
            temp = np.c_[original,next_layer]
            
            
            #print("=======")
            
            for u,p in zip(original,next_layer):
                line = plt.Line2D([(i*scale),(i+1)*scale],[u*scale,p*scale],color = "grey", lw = 0.3)
                plt.gca().add_line(line) 
            
            # for u in temp:
            #     #print(i,i+1)
            #     #print(u[0],u[1])
                
            #     line = plt.Line2D([i,i+1],[u[0],u[1]])
                
            #      
    for i in range(0,len(node_locations)):
        layer = node_locations[i]
        if i == 0:
            colour = 'green'
        elif i == len(node_locations)-1:
            colour = 'red'
        else:
            colour = 'black'
        plt.scatter(np.full(len(layer),i)*scale,layer*scale,zorder=10,color = colour,s=20/scale)
            
            
            # circle1 = plt.Circle((i,layer[o]),0.1,color = 'r')
    
            # ax.add_patch(circle1)        
    
    
    plt.xlim([-1*scale, (i+1)*scale])
    plt.ylim([-1*scale, (max_width+3)*scale])
    plt.axis('off')
    
    plt.text((0-1)*scale,(max_width)*scale,"Input layer",color = 'green',fontsize = 20)
    plt.text((0+1)*scale,(max_width)*scale,"Hidden layers, "+str(hidden_layer_sizes),fontsize = 20)
    plt.text((len(layer_widths)-1)*scale,(max_width)*scale,"Output Layer",color='red',fontsize = 20)
    
    plt.text((0.75)*scale,(max_width+2)*scale,"Neural network diagram",fontsize = 30)
    
    
    
    
    fig.set_size_inches((max_width+3)*scale*10,(i+1)*scale*10)
    #plt.savefig('myfigure_100.png', dpi=300)
    plt.show()
    #plt.rcParams["figure.figsize"] = ((i+1),(max_width+1)*2)   
    
    
            
            
            
           
            
    
    
    
    #print(np.array(node_locations))    




#print(np.c_[np.array([1,1,1,1,1]),np.array([0,1,2,3,4])]) 

In [ ]:
#Try out different sizes of layers
NeuralNetworkPlot(6,(3,4),1) 
#                 ^input nodes
#                     ^hidden nodes
#                         ^output nodes

### Section 2 quiz ###

In [ ]:
#@title
print("Which of the following is NOT a 'black box' machine learning model?")
out = widgets.Dropdown(options=[('',0),('Random forest',0),('Neural network',0),('Decision tree',1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In random forest machine learning models, how can I reduce overfitting?")
out = widgets.Dropdown(options=[('',0),('Increase each decision tree\'s maximum depth',0),('Increase the number of decision trees in the forest',1),('Decrease the number of variables being used',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("I want to design a machine learning model which is"+ 
      " capable of sorting 10,000 \n images a day based on their"+
      " average colour, average brightness, resolution in \n DPI "+
      "(dots per inch) and the number of red pixels contained "+
      "within them. My \n model would need to split the data into "+
      "seven categories. How many dimensions \n would I need to "+
      "represent the data hyperspace?")
out = widgets.Dropdown(options=[('',0),('2',0),('4',1),('7',0),('10000',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("You decide to train a neural network on your system which uses an Nvidia \n"+
      "graphics card. When you decide to switch to an identical system, except that \n"+
      "it uses an Intel graphics card, you find that the neural network no longer works \n"+
      "as expected. Why?")
out = widgets.Dropdown(options=[('',0),('Neural networks can be hardware dependent',1),('You can\'t use neural networks on Intel graphics cards',0),('You made a programming mistake',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

## 3: Application ##

We will now use these concepts to create two binary classifiers using a random forest and a neural network. We will train them using the same Monte Carlo data used in Workbook 2 of this workbook series.

### 3.1: Pre-processing ###

As said in Section 2.2, we shall be using seven properties of  the collision events as the features for the machine learning models:

<table>
<thead>
<tr><th>Leading lepton's transverse momentum</th><th>Sub-leading lepton's transverse momentum</th><th>$m_{ll}$</th><th>$E_{Tmiss}$</th><th>$\Delta{}\phi{}$</th><th>Fractional transverse momentum difference</th><th>$E_{Tmiss}$/$H_T$</th></tr>
</thead>
</table>


In [ ]:
# identify the properties we want
ML_inputs = ['totalWeight',
             'lead_lep_pt', 
             'sublead_lep_pt', 
             'mll',
             'ETmiss', 
             'dphi_pTll_ETmiss', 
             'fractional_pT_difference',
             'ETmiss_over_HT',
             'processID'] #copy over process ID but will not be used in ML training

We shall start the pre-processing by creating a dictionary of data to be used by the machine learning models. We will start by copying over all the event data but only while disregarding any event properties we don't care about. **Note :** You will also notice that "totalWeight" is also included in the dictionary. It is not because this is directly used as a feature in machine learning models. Rather, this is to account for the [selection effects](https://docs.google.com/document/d/1B_bzcKqTnBBZ-0K02aurW9o8FgfWlmEO-PxuP-f4ves/edit) of Monte Carlo generated data. Please keep an eye on where "totalWeight" makes appearances throughout this workbook, since this is used to properly scale our analysis and evaluation of the trained models.  

In [ ]:
#This dictionary will hold all the event properties we will feed into the ML model
ML_data={}

#Again looping through all the processes 
for process in processes:
    ML_data[process] = All_processes[process][ML_inputs].copy()
    #This line makes entrys in ML_data by copying all the ML inputs from All_processes
    #see pandas indexing for more infomation https://pandas.pydata.org/docs/user_guide/indexing.html

As said earlier before training our machine learning models, we must prepare the data so the computer can make sense of it. We start doing this by creating two data sets:

* **$X$ data set**: this will hold all the input data for the machine learning model. This is done by selecting all the events from every process, then filtering the properties/features we want and combining them into a larger dataset. Note that the order of this data set is important.



In [ ]:
#We shall be using scikit-learn to split the data into the training and test sets
from sklearn.model_selection import train_test_split
#This function will split the data and suffle it, ratio of test to train can be changed here as well

def splitter(X, y, test_size = 0.33, random_state = seed_value):
  X_train_WID, X_test_WID, y_train, y_test = train_test_split(X, y, 
                                                              test_size = 0.33, 
                                                              random_state = seed_value)


  # Save the process IDs of the events from the train and test data sets 
  X_train_ID = X_train_WID[:,-1]
  X_test_ID = X_test_WID[:,-1]
  # Take the process IDs off so they dont interfere
  X_train_t = np.delete(X_train_WID,-1,1)
  X_test_t = np.delete(X_test_WID,-1,1)


  # Save the weights of the events from the train and test data sets
  X_train_W = X_train_t[:,0]
  X_test_W = X_test_t[:,0]
  # take the weights off so they dont interfere
  X_train = np.delete(X_train_t,0,1)
  X_test = np.delete(X_test_t,0,1)

  return X_train, X_train_W, y_train, X_test, X_test_W, y_test

In [ ]:
#We will now create the X data set ===========================================
#Looping through each process and appending each to a numpy array that will hold 
#all the event data
all_ML_data = []
for process in processes:
    all_ML_data.append(ML_data[process])

X = np.concatenate(all_ML_data)
#=============================================================================

* **$y$ dataset**: this is a list of 1's and 0's corresponding to signal and background that lines up with the $X$ data set.

In [ ]:
#We will now create the y data set: ==========================================
#Again looping through each process process and appending a set of either 1 or 0 
#depending if the process is background or not
all_ML_classes = []

for process in processes: 
    process_data_size = ML_data[process].shape[0] #set number of events in process
    if 'DM' in process:
        all_ML_classes.append(np.ones(process_data_size)) #1's for signal
    else:
        all_ML_classes.append(np.zeros(process_data_size)) #0's for background

y = np.concatenate(all_ML_classes)
#=============================================================================

The diagram below shows the two data sets we have now created if each process has four data entries (obviously this is not actually the case):
![data_diagram](https://lh3.googleusercontent.com/Q-oatcdBOqaxC4Oh7eIThv9VZCbySHq_xuUZjIrg46css0NTVqWUTvRw6BsHf8K8-f4=w2400)

In machine learning training, it is important to have training and testing sets of data. 

> Isn’t it the dream of every student to get very similar exam questions to ones they practiced with? It is easy to score well when you are familiarized with the problems being asked. To guarantee a fair test, teachers make sure to split their bank of questions into a training and a testing set. As such, students will have plenty to practice but still receive unseen questions on the test. The same applies to machine learning!

If the machine learning model is trained on all the data, then tested on all the data the results will not give us any understanding on the effectiveness of the model as it is biased to test it on what it was trained with.

To program this split of training and testing data, we shall simply feed in the **$X$** and **$y$** data sets into a function from the scikit-learn module that does it for us. Here, we will reserve a third of the data for testing with the remainder for training.


In [ ]:
#We shall be using scikit-learn to split the data into the training and test sets
from sklearn.model_selection import train_test_split
#This function will split the data and suffle it, ratio of test to train can be changed here as well
X_train_WID, X_test_WID, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = seed_value)


# Save the process IDs of the events from the train and test data sets 
X_train_ID = X_train_WID[:,-1]
X_test_ID = X_test_WID[:,-1]
# Take the process IDs off so they dont interfere
X_train_t = np.delete(X_train_WID,-1,1)
X_test_t = np.delete(X_test_WID,-1,1)


# Save the weights of the events from the train and test data sets
X_train_W = X_train_t[:,0]
X_test_W = X_test_t[:,0]
# take the weights off so they dont interfere
X_train = np.delete(X_train_t,0,1)
X_test = np.delete(X_test_t,0,1)

These data sets must now be normalised. This is important for the neural network model to converge. This is done again using scikit-learn. 
The importance of normalisation isn't obvious, but is needed. You may also think of it akin to standardisation, because it brings the mean average of data inputs close to some standard value (e.g. 0). This helps to speed up the optimisation of a neural network and reduce 'issues' along the way. These issues are intimately related to gradient descent (see Section 4.2.1) and the way in which a neural network is trained. For now, an analogy may be useful:

> Let's say you are trained to work in a factory screwing various nuts and bolts into holes in pieces of furniture. If you are trained to work with nuts and bolts which are all of a similar size and weight, your musucle memory will help you to perform this task quicker; but if you are trained to work with nuts and bolts which vary irratically in this respect, it can be harder for you to optimise your way of working to easily complete your task. Thus, the factory decide to use a standard range of parts for you to use and you are happier.

By standardising/normalising the inputs in machine learning, the machinary behind neural networks can behave in a more "predictable" way, thus improving the outcome.


<html>
<details>
<summary style="color:orange; font-weight: bold;">For more information</summary>
please have a look [here](https://towardsdatascience.com/why-data-should-be-normalized-before-training-a-neural-network-c626b7f66c7d), [here](https://arxiv.org/pdf/1805.11604.pdf) and [here](https://medium.com/techspace-usict/normalization-techniques-in-deep-neural-networks-9121bf100d8).

In [ ]:
#import the needed function from scikit-learn
from sklearn.preprocessing import StandardScaler

#call the scaler opject then fit it to the X_train data
scaler = StandardScaler()
scaler.fit(X_train)

#Now scale/normalise both of the X data sets by transforming them with the scaler we just created
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

We now have the needed data to train our machine learning models.

### Section 3 quiz ###

In [ ]:
#@title
print("Why must there be independent training and testing datasets when developing machine learning models?")
out = widgets.Dropdown(options=[('',0),('To assess model accuracy',0),('To assess overfitting or underfitting',0),('Both',1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In the above code, why is the \"processID\" deleted from the data"+
      "before the data are used in training or testing?")
out = widgets.Dropdown(options=[('',0),('We do not want the "processID" to be treated as an input variable',1),('We simply want to cut down on memory usage',0),('The MC data don\'t have processIDs to use',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("I decide to train a machine learning model to classify objects as \"round\" or \"not round\".\n"+
      "I choose to use two class labels: 0 and 1. If my objective is to select objects which are \n"+
      "\"round\" and to discard those which are \"not round\", does it matter which label I use for \n"+
      "which category?")
out = widgets.Dropdown(options=[('',0),('Yes',0),('No',1),('It depends',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

## 4: Model training ##


Now that we understand and have prepared the data and features that we are using, we will now go through the process of training our machine learning models. As a reminder, training will involve teaching the algorithms with data which it will attempt to categorise between DM signal and background. As the data is labelled to indicate the "true" signal and background, this is **supervised learning** in which the labels provide feedback for how well the machine learning models are doing.  




### 4.1: Random forest (RF) ###

We shall start by training the random forest classifier. This is completed on a tree-by-tree basis. Each decision tree training is completed using the "classification and regression tree" (CART) algorithm, then ensemble learning is used to make final predictions from all the separate tree predictions.

#### 4.1.2: Training the RF #### 

We shall be training the random forest using scikit-learn. This module will give us the options to change the model’s hyperparameters before training:

*   **Criterion** : The function used to measure the quality of a split; it determines if a node is now a leaf node. 
*   **Bootstrap** : Bootstrap or not? (See Section 2.3)
*   **Max depth** : The maximum depth of every decision tree.
*   **Max features** : Number of features to consider for each decision tree. 
*   **No. of estimators** : The number of decision trees in the forest.
*   **Random state** : Controls both the randomness of the bootstrapping of the samples used when building trees, and the sampling of the features to consider when looking for the best split at each node.

Let's train the random forest classifer with these hyperparameters:

In [ ]:
# import the random forest Class
from sklearn.ensemble import RandomForestClassifier
# initialize the classifer with the wanted hyperparameters
RF_clf = RandomForestClassifier(criterion = 'gini', 
                                bootstrap = True,       
                                max_depth = 4,  
                                max_features = "sqrt",           
                                n_estimators = 500,         
                                random_state = seed_value, 
                                verbose = 1, #
                                n_jobs = -1) #

# train the classifer (use normalised X data to help converge)
RF_clf.fit(X_train_scaled,y_train) 

### 4.2: Neural network (NN) ###

Now we will be training the neural network classifier. To begin, the neural network is randomly seeded, then something called the loss function is calculated by comparing predictions (on the training data) to the expected values. Next, we perform something called gradient descent to minimise this loss function which is done through backpropagation - adjustment of weights for each neuron, layer-by-layer, to improve the prediction. You may read more about the training in the next advanced section however, it is not necessary to continue the notebook (and will not be in the quiz).


#### 4.2.1: Advanced section: NN training####

Insert YouTube videos

"Gradient descent is an iterative operation which seeks to find local minima or maxima of a function" may be one such way of defining it, but this may be a lot to unpack. Instead, it may be more convenient to use an analogy: gradient descent is analogous to the whole process of hiking down the mountain and it is related to the steepness of the loss function. In more technical terms, it is the iterative process of calculating the current value of the loss function (scouting) and then altering the inner parameters of the NN by a step down in the loss function (stepping). The NN repeats this process through multiple iterations, each time decreasing the step size (field of view decreases) until it reaches a model optimized for successfully classifying signal and background events (a city is reached).

In the animation below, the black dot represents the position of a neural network as it descends along the gradient by a definite step size. Just like you might see in a topographic map, the gradient of the loss function is colour-coded in a mesh where red are the peaks and blue are the valleys. Note how the descent slows down as the gradient flattens. Did your hiking adventure look like that?

![Gradient Descent Diagram](https://drive.google.com/uc?export=view&id=1YeUNA9oDVNyPxX_Re1lIx1Z3ZIBm24Nv)

You may recall that the normalisation is linked with gradient descent. In more precise language, normalisation/standardisation is done to make the gradient descent optimisation work faster. If data is not normalised, there is the chance that gradient descent is less likely to converge to a solution for a loss function (e.g. you are hiking along a "mountain" which looks rather flat for miles and miles around and are unable to easily see a path to the lowest point; or you are hiking over such a small mound of a mountain - say an anthill - that your stride is great enough that you step right over the mound without seeing it!!). 

#### 4.2.2: Training the NN #### 

We shall also be training the neural network using scikit-learn. Again, we have the option to change the model’s hyperparameters before training:

*   **Solver** : The solver used in weight optimization (see Section 4.2.1).
*   **Activation** : Activation function for the hidden layer  (see Section 4.2.1).
*   **Hidden layer sizes** : Set the number of hidden layers and the number of nodes in each one.
*   **Shuffle** : Whether to shuffle samples in each iteration or not.
*   **Random state** : Determines random number generation for seeding weights and bias initialisation.



Let's train the neural network classifer with these hyperparameters:

In [ ]:
# import the multi-layer perceptron Class
from sklearn.neural_network import MLPClassifier
# initialize the classifer with the wanted hyperparameters
NN_clf = MLPClassifier(solver = 'adam',
                        activation ='relu',
                        hidden_layer_sizes = (100,5),
                        shuffle = True,
                        random_state = seed_value,
                       
                        verbose=1)

# train the classifer (use normalised X data to help converge)
NN_clf.fit(X_train_scaled, y_train) 

### 4.3: Prediction sneak peek ###

Now that we have trained the classifiers, lets briefly experiment with prediction. First of all, we will pick a random event object from the test data set:

In [ ]:
from numpy.random import randint
#get a random number to sample random object 
rand_num = randint(0,X_test.shape[0])

#Take a random object from x test
event_object = X_test[rand_num]
scaled_event_object = X_test_scaled[rand_num]

Let's find out if this event is signal or background:

In [ ]:
#Is this event signal or background? 
event_object_label =  y_test[rand_num]

#Print features of object and label
print(event_object,event_object_label)


So now we know what this event is actually classified as, let’s see if the classifiers correctly predict what this event is (signal or background). We can do this using the **predict($X$)** method in scikit-learn, this returns the predicted class of an input sample.


In [ ]:
#make both into 2d arrays so .predict works
event_object = np.array([event_object])
scaled_event_object = np.array([scaled_event_object])

print(event_object)

#Perdict using both classifers
RF_Perdiction = RF_clf.predict(event_object)
NN_Perdiction = NN_clf.predict(scaled_event_object)

#print predictions
print("Random forest prediction:",RF_Perdiction)
print("Neural network prediction:",NN_Perdiction)

This class prediction is found from the prediction probabilities of being signal and of being background. We can get these probabilities by using the **predict_log_proba($X$)** method in scikit-learn which acts on the classifier:

In [ ]:
#Find the probability of predictions
RF_Perdiction_prob =  RF_clf.predict_proba(event_object)
NN_Perdiction_prob = NN_clf.predict_proba(scaled_event_object)

#print probability of predictions
print("Random forest prediction probability:",RF_Perdiction_prob)
print("Neural network prediction probability:",NN_Perdiction_prob)


This method returns two probabilities in an array with index 0 holding the background probability and index 1 holding the signal probability. We are most interested in the probability of an event being signal. These probabilities are much more use to us then just the class label as we can study the range of probabilities to understand how our classifiers are working.

In the next section we will look at these predicted probabilities (also considered the *response* of the machine learning model) for the whole test data set $X$. We will see that the full responses produce distributions which can be analysed, like what was done in Workbook 2.


### Section 4 quiz ###

In [ ]:
#@title
print("When training a neural network, how does it optimise itself?")
out = widgets.Dropdown(options=[('',0),('By maximising a "gain" function',0),
                                ('By stabilising an output',0),
                                ('By minimising a "loss" function',1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("Which of the following is NOT a hyperparameter of a neural network?")
out = widgets.Dropdown(options=[('',0),('Hidden layer sizes',0),
                                ('Verbose',1),
                                ('Activation',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In training the random forest, which of the following hyperparameters was responsible \n"+
      "for each decision tree being trained on a random sample of the training data with replacement?")
out = widgets.Dropdown(options=[('',0),('Random state',0),
                                ('Bootstrap',1),
                                ('No. of estimators',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

## 5: Prediction and model evaluation ##

Now that we have trained the classifiers, we can use them to predict if an event is background or signal.

We must also evaluate these models to understand how well they are working and give us the chance to optimise the machine learning models' responses. We will perfect the classifiers using the Monte Carlo data, then we will be able to use it in practice with real experimental data. 

 

### 5.1: Model analysis ###

Analysis methods include:
*   Plotting ML model response for signal and background
*   ROC curve
*   Overfitting check
*   KS test

This section will go through these methods, and you will be given a chance to optimise the classifiers for the best and most accurate prediction.


#### 5.1.1: Model response ####

Looking at the response of the classifiers using the test data we see how the model is responding to the signal and background differently. As was said in Section 4.3, we will look at the predictions for the whole $X$ test data set. When we do this, we will see a distribution of responses over the whole data set. The goal for the coming sections will be to optimise this response to clearly see two distributions in this data - one for signal and one for background. This indicates new physics.

How would we plot this? If we take the $x$-axis of a plot to be the predicted probability that an object is signal, then we plotted all the objects in a sample as two separate histograms (one for signal labelled objects and one for background labelled) on the same axis, we can see the distributions. We can do this as we know beforehand which events are background and signal (the $X$ test set corresponds with $y$ test labels). Note this would not be possible with real, non-Monte Carlo data as it is unlabelled. Let’s consider a perfect classifier, the plot would look like:



![out_1](https://lh6.googleusercontent.com/TiZwRuPDGvuDb6jZ2H2ymbyplcCBr1DCs-f45ufKFroxFpsxUUqXRymtXPJ-ezYyddU=w2400)

In reality the classifier, is not perfect, but a strong separation of signal from the background could look like this:
![output_2](https://lh5.googleusercontent.com/Vfn64iGQDJVxQvFFQEkol-DGgCGUQGLyWIfk7GLcOdTShAFTO8xef1ZkTgOLpwp9wGs=w2400)

A real example of this from the ATLAS experiment can be seen below. This shows two clear distributions for the total background (blue) and the Higgs signal (dotted red). NOTE: the horizontal axis here is NN output not the prediction, but generally these can be considered as the same (just with a different horizontal axis range). The Higgs signal considered here is a rather specific process which (crucially) involves a Higgs boson.

![atlasNNoutput](https://lh5.googleusercontent.com/9J_6RG1mf8jIg5Uy2J4zVTmvTXqZE-0jHz4KO0OV9jne3Rl93HbSdUEpNg9srK74yrc=w2400)

The two distributions clearly show different trends in how the neural network responds, suggesting it is a useful NN for isolating a signal. This neural network was developed in work which sought to find evidence of a Higgs boson by isolating the signal for this specific Higgs process. This graph shows that a cut (more in Section 5.2) on the NN output between $0.0$ and $0.5$ could strongly select for the Higgs process. We will be looking to use our ML models in a similar way to find dark matter!!




Now let’s plot the response of our classifiers. We will plot two histograms - orange for signal and blue for background. While the $y$-axis of response plots can be expressed using a variety of units, the plots we will be creating will be histograms for the **frequency** with which the classifier outputs some signal probability between $0$ and $1$. We shall write a function to plot the output of any classifier for the $X$ data we give it:

In [ ]:
BINS = 16

def plot_response(clf,Title,sig,sig_weights,bkg,bkg_weights,n_bins,Log):
    """
    This function plots the response of a classifer on signal and background 
    Monte Carlo events


    Parameters
    ----------
    fig : TYPE
        DESCRIPTION.
    clf : TYPE
        DESCRIPTION.
    Title : TYPE
        DESCRIPTION.
    sig : TYPE
        DESCRIPTION.
    sig_weights : TYPE
        DESCRIPTION.
    bkg : TYPE
        DESCRIPTION.
    bkg_weights : TYPE
        DESCRIPTION.
    n_bins : TYPE
        DESCRIPTION.
    index : TYPE
        DESCRIPTION.
    xAxis : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    

    
    
    # Get response (in form of probabilities) of signal and background 
    sig_prob = clf.predict_proba(sig)[:, 1]
    bkg_prob = clf.predict_proba(bkg)[:, 1]
    # find min and max for plot
    d_min = min(sig_prob.min(), bkg_prob.min())
    d_max = max(sig_prob.max(), bkg_prob.max())
    
    
    
    # plot background response (make orange and translucent)
    plt.hist(bkg_prob,
             bins = n_bins,
             range = (d_min,d_max),
             color = 'tab:blue', 
             label = 'bkg test',
             alpha = 0.4, 
             density = False,
             weights = bkg_weights,
             log = Log,
             histtype="stepfilled",
             edgecolor='tab:blue',
             linewidth=5)
    
    
    # plot signal response (make blue and translucent)
    plt.hist(sig_prob,
             bins = n_bins,
             range = (d_min,d_max),
             color = 'tab:orange', 
             label = 'sig test', 
             alpha = 0.4, 
             density = False,
             weights = sig_weights,
             log = Log,
             histtype = "stepfilled",
             edgecolor ='tab:orange',
             linewidth = 5)

    # Set up plot
    plt.title(Title)
    plt.xlabel("Response (0 = background, 1 = signal)")
    plt.ylabel("Frequency")
    plt.legend()
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    plt.show()
    

    
    return

To call this function properly, we must sort the $X$ test and training data into background and signal data sets; further, we must sort the $X$ data's **weights** too (remember that since the data here are Monte Carlo, we MUST use the weights to **rescale** the data so that we can plot the frequencies as if they were real data):

In [ ]:
# Sort signal data into new arrays one for X vaules and one for the weights
sig_test     =  X_test_scaled[y_test >= 0.5]
sig_test_W   =       X_test_W[y_test >= 0.5]
sig_train    =  X_train_scaled[y_train >= 0.5]
sig_train_W  =       X_train_W[y_train >= 0.5]
# Sort background data into new arrays one for X vaules and one for the weights
bkg_test     =  X_test_scaled[y_test <= 0.5]
bkg_test_W   =       X_test_W[y_test <= 0.5]
bkg_train    =  X_train_scaled[y_train <= 0.5]
bkg_train_W  =       X_train_W[y_train <= 0.5]

Let's call the function and plot the response of the classifers, first for the random forest:

In [ ]:
plot_response(RF_clf, 
              "Random forest response", 
              
              sig_test, 
              sig_test_W,
              bkg_test,
              bkg_test_W,
              
              BINS,
              False)

It is hard to see the signal as dark matter is such a rare process. So let's set the $y$-axis to logarithmic scaling:

In [ ]:
plot_response(RF_clf, 
              "Random forest response", 
              
              sig_test, 
              sig_test_W,
              bkg_test,
              bkg_test_W,
              
              BINS,
              True)

That is much better - we can see the distributions. Now let's do the same for the neural network:

In [ ]:
plot_response(NN_clf, 
              "Neural network response", 
              
              sig_test, 
              sig_test_W,
              bkg_test,
              bkg_test_W, 
              
              BINS,
              True)

At the start of Section 5.1.1, we outlined some general, if optimisitic, response plots which showed a good degree of separation between some background and some signal. In contrast, the plots which we have produced show a less distinct separation between DM signal and background in the MC data. This doesn't mean that the neural network or random forest are not doing anything to classify the data. Rather it is simply because we are trying to find a very tricky DM signal (which remember is similar enough to our background such that a cut-based optimisation is not enough). 

Instead, we see that the background and signal responses follow different responses;

* For the random forest, the response is rather uniform for the signal data, whereas the background response is peaks towards $0.00$ and decreases for higher probabilities;

* For the neural network, the signal response is less uniform than for the neural network but is spread out over a larger range than the random forest; the background response peaks towards $0.00$ and decreases rapidly.

From our intermediate results, you may be able to make a guess as to which machine learning model produces a better separation between background and signal - neural network or random forest?

####  5.1.2: ROC curve ####


To understand how well the machine learning model is working, we can look at the ROC curve. To do this, let's first introduce some new metrics: <font color='green'>TP</font> (number of true positives), <font color='red'>FN</font> (number of false negatives), <font color='red'>FP</font> (number false positives) and <font color='green'>TN</font> (number of true negatives). These are defined as the following:

*   TP: Signal events correctly identified as signal events;
*   FN: Signal events incorrectly identified as background events;
*   FP: Background events incorrectly identified as signal events;
*   TN: Background events correctly identified as background events.

As you may have realised in this case, a background event is considered negative while a signal event is considered positive. 

The box in the image below shows the definition of these metrics. The columns divide objects predicted as signal (left) and background (right) while the rows divide objects that are actually signal (top) and background (bottom).

From these metrics we can construct two values to describe the effectiveness of our classifiers: true positive rate and false positive rate. Below these values are explained:

![ROC](https://lh4.googleusercontent.com/dRQIT12SO5RR4paENExHGImZuJXMV49OR6Ezk8NtmclBLDnmJUBdLmlnq9fIr-d45eg=w2400)

Now we have these two values we can optimize them for the best classification. Let’s think back to Section 5.1.1 where we plotted the response of our models in the form of two histograms on the same plot to show the distributions:
![NN_response](https://lh3.googleusercontent.com/p8MOLHdqZSGFAaBsmpxTjwXm-BwRK4lZtlY_SPSO4rK_5JgCISCJlTuuJ81aGHhWQDA=w2400)

Consider a vertical line on this plot that represents a threshold. Everything to the right of that vertical line is now classified as a signal event, and everything to the left a background event. We can move this line to the left or right in turn changing the threshold. Doing this changes the TP, FN, FP and TN numbers and thus in turn changes the TPR and FPR. If we change this threshold from 0 to 1 (going from left to right) we can plot TPR and FPR as they change. This plot is called the 'receiver operating characteristic' (ROC) curve. With TPR as the $y$-axis and FPR as the $x$-axis, the plot below shows (as well as other lines) an example of a neural networks ROC curve (blue line):

![rocCurve](https://lh6.googleusercontent.com/1hC1foz56Jr5pLrluF0sRyaOKR5Nr0Oj0mgd-8ls2OMOHCESigFL7DcK9paD888yScQ=w2400)


*   **Perfect classifier** : A perfect classifier will 100% correctly classify signal from background;
*   **Random classifier** : A random classifier will randomly classify signal form background;
*   **NN classifier** : An example of a decent classifier.

The goal is to minimise FPR and maximise TPR, or in other words, to get the curve as far as possible to the top left hand corner.


Now let’s plot the ROC curves for both our classifiers. Luckily for us, scikit-learn has a built-in function that does this for us. The **roc_curve** function takes in the $y$ test data, the array of thresholds (classifier response probabilities) and outputs corresponding FPRs and TPRs:

In [ ]:
# import the needed modules to analyse the classifers
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score

# get predictions for X test data
y_pred_RF = RF_clf.predict(X_test_scaled)
y_pred_NN = NN_clf.predict(X_test_scaled)


Let’s now define a function to plot the ROC curve for both a classifer and a random classifier:

In [ ]:
def calc_roc(clf, 
             X, 
             X_weight, 
             y,
             Title):
    """
    This function calculates the ROC curve for a given classifer then plots it.

    """
    
    # Get the prediction of the classifer
    y_prob = clf.predict_proba(X)[:,1]
    
    # Use the scikit-learn function to calulate the FPR and TPR vaules corresponding
    # to the therssholds. The line after that finds the area under the ROC curve
    fpr, tpr, thresholds = roc_curve(y ,y_prob, sample_weight = X_weight)
    # roc_auc = roc_auc_score(y ,y_prob, sample_weight = X_weight,labels=[0,1])
    roc_auc = 1
    
    # set up the plot
    plt.figure()
    plt.plot(fpr,tpr,color='tab:orange', lw=2, label='ROC Curve (area = %0.2f)'%roc_auc)
    plt.plot([1,0],[1,0], color='tab:blue', lw=2, linestyle='--', label='Random classifier')
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(Title)
    plt.legend() 
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    
    
    plt.show()
    
    # return all the calculated vaules for use later
    return roc_auc, fpr, tpr, thresholds

Now let's call this function for both classifiers:

In [ ]:
# plot the ROC curves for the neural network and the random forest and get their  
# corresponding FPR and TPR vaules for later
auc_NN, fpr_NN, tpr_NN, thresholds_NN = calc_roc(NN_clf, X_test_scaled, X_test_W, y_test, "ROC curve for the neural network classifer")
auc_RF, fpr_RF, tpr_RF, thresholds_RF = calc_roc(RF_clf, X_test_scaled, X_test_W, y_test, "ROC curve for the random forest classifer")

We can see that both the random forest and the neural network behave better than random classification. However, how much better than random are they? The best way to answer this question is to look at how close each of the ROC curves gets to touching the upper left corner (which represents a perfect classifier with TPR=$1$ and FPR=$0$). Although the difference is subtle, which of the two models is closer to this target, showing it is a stronger classifier?

#### 5.1.3: Overfitting + KS test ####

As supervised machine learning involves training a classifier on example data, you can run into the problem of the model fitting 'too well' to the example data, such that it loses generality and will no longer predict unlabelled data correctly.

We could also consider the case where we train our model on a smaller volume of objects than we test on. This could result in overfitting - the classifier fitting too well to this smaller data set. This makes the model too specific, so that when new objects are introduced, they are misclassified. The diagram below shows this issue:


![overfit](https://lh6.googleusercontent.com/8L-Djt0LQTs_gRM9ZpLY4nM3nfbBp9J74osbv5EJjOngdDwu7yPSPBpUQJ_PEuXS7BE=w2400)



You can imagine this threshold to be akin to the difference between understanding and memorising how to solve a particular set of questions. At first, you might just get the overall idea but still incorrectly answer the majority of questions (underfitting). With more practice, you may reach a point where you are comfortable with the content and can correctly answer most of the questions (good fitting). Here is where you should stop, congratulate yourself, and move onto another fresh set of practice questions. If you keep practising even further with the same set, however, you might memorise the answers and end up not getting a grasp of their reasoning at all (overfitting).

Certain machine learning models are susceptible to overfitting, such as decision trees, but generally, all models can struggle with this. There are many ways to overcome this:
* Changing the test/train data ratio or increasing the volume of training data;
* Reducing the model complexity by changing their hyperparameters:
* * This may be reducing the maximum depth of decision trees in a random forest or increasing the number of trees;
* * Or in a neural network this may be reducing the size of your hidden layers;
* And generally this may be reducing the number of features in the input data which are used by the ML model. You may think of this as akin to the aphorism, **"if you go looking for trouble, it will come"**. More specifically, if too many data features are used, it may be possible for the model to "spot" patterns which are not physical, but rather an artifact of the specific data that is used.

We can see if our model is overfitting be comparing the response of the $X$ test data and $X$ train data. Ideally, the response to each dataset should be the same, but if these responses are not similar, it is a sign that the classifier has underfitted or overfitted to the training data.

</br>

We can statistically compare these responses by using the two-sample Kolmogorov–Smirnov test, also known as a KS test. This is a “goodness of fit test” which compares two distributions and tests if they have the same distribution. We will compare the test data response to the train data response. The KS test returns a metric called the Kolmogorov–Smirnov statistic and a p-value (number describing how likely the test data would have occurred under the null hypothesis of this statistical test). The latter of the two is often easier for drawing conclusions.
* Kolmogorov–Smirnov statistic : A metric which assesses the match or mismatch between two distributions by quantifying the distance between them. It assumes the **null hypothesis $H_0$** that they come from the same underlying distribution. Smaller values show stronger agreement.
* P-value : the probability that $H_0$ is true; i.e. the test and train responses show the same distribution. Often, a $5\%-95\%$ rule is used. If the p-value is significantly greater than a $0.05$ threshold, our **$H_0$** can be accepted and the **alternative hypothesis $H_1$** can be rejected; i.e. the train and test responses show good agreement rather than severe overfitting. However, if the p-value is too close to $1.00$, we may have to double check our work as this result is suspiciously good.

</br>

Let’s now see if our models are overfitted. To do this, we will plot the response of the test data against the train data. Let's program a function to do just that:

In [ ]:
# import the needed module to preform a Kolmogorov–Smirnov test (ks test)
from scipy.stats import ks_2samp


def plot_overfittingCheck(clf, Title,                        
                          sig_train, sig_train_weights, sig_test, sig_test_weights,       
                          bkg_train, bkg_train_weights, bkg_test, bkg_test_weights,
                          n_bins, test_size=0.33):
    """
    This funtion plots the overfitting check plots, while also returning the results
    of a Kolmogorov–Smirnov test comparing the test data response and train data response

    """
    

    
    # get the predictions (in form of probabilities) for background and 
    # signal for the train data set and the test data set
    sig_train_prob = clf.predict_proba(sig_train)[:, 1]
    bkg_train_prob = clf.predict_proba(bkg_train)[:, 1]
    
    sig_test_prob = clf.predict_proba(sig_test)[:, 1]
    bkg_test_prob = clf.predict_proba(bkg_test)[:, 1]
    
    # find min and max for plot
    d_min = min(sig_train_prob.min(), bkg_train_prob.min())
    d_max = max(sig_train_prob.max(), bkg_train_prob.max())
    
    
    
    
    # plot the background response of the train data set (make orange and translucent)
    bkg_tr, bins, _ = plt.hist(bkg_train_prob,
                               bins=n_bins,
                               range=(d_min,d_max), 
                               color='tab:blue', 
                               label='bkg train',
                               alpha=0.4,
                               weights = bkg_train_weights/(1-test_size),
                               log = True,
                               histtype="stepfilled",
                               edgecolor='tab:blue',
                               linewidth=5)
    
    # plot the signal response of the train data set (make blue and translucent)
    sig_tr, bins, _ = plt.hist(sig_train_prob,
                              bins=n_bins,
                              range=(d_min,d_max), 
                              color='tab:orange', 
                              label='sig train', 
                              alpha=0.4,
                              weights = sig_train_weights/(1-test_size),
                              log = True,
                              histtype="stepfilled",
                              edgecolor='tab:orange',
                              linewidth=5)
    
    

    # find bin centres
    bin_centers = (bins[:-1] + bins[1:])/2
    # use numpy histograms to find heights of histogram bars for the test data set
    sig_te, _ = np.histogram(sig_test_prob, bins = bins, weights = sig_test_weights/test_size)
    bkg_te, _ = np.histogram(bkg_test_prob, bins = bins, weights = bkg_test_weights/test_size)
    
    # plot the test background and signal histograms as points 
    plt.plot(bin_centers,bkg_te, 'o', c = 'tab:blue', label = 'bkg test', alpha = 0.9, markeredgecolor = 'k')
    plt.plot(bin_centers,sig_te, 'o', c = 'tab:orange', label = 'sig test', alpha = 0.9, markeredgecolor = 'k')
    
    # set up the plot
    plt.title(Title)
    plt.xlabel("Response (0 = background, 1 = signal)")
    plt.ylabel("Normalised frequency")
    plt.yscale("log")
    plt.legend()
    
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    
    plt.show()
    
    # return the Kolmogorov–Smirnov test results
    return ks_2samp(sig_tr, sig_te)

It may be worth highlighting that the test and training data are different sizes based on the way how the Monte Carlo data were split in Section 3. This is important since the height of a histogram bar in the training data plots would be expected to be smaller than the height of a test data histogram bar. To get around this and to allow us to make visual bin-by-bin comparisons between the test and training data, the function **rescales** both the test and training data to have the same total frequency.

Let’s call this function now for the random forest:

In [ ]:
# preform overfitting check for both classifers and get Kolmogorov–Smirnov test results   
ksTest_RF = plot_overfittingCheck(RF_clf, 
                                  "Random forest overfitting check", 
                                  sig_train, 
                                  sig_train_W,
                                  sig_test,
                                  sig_test_W,                                  
                                  bkg_train, 
                                  bkg_train_W,
                                  bkg_test, 
                                  bkg_test_W, 
                                  BINS)

In [ ]:
print(ksTest_RF)

And the same for the neural network:

In [ ]:
ksTest_NN = plot_overfittingCheck(NN_clf, 
                                  "Neural network overfitting check", 
                                  sig_train, 
                                  sig_train_W,
                                  sig_test,
                                  sig_test_W,                                  
                                  bkg_train, 
                                  bkg_train_W,
                                  bkg_test, 
                                  bkg_test_W, 
                                  BINS)

In [ ]:
print(ksTest_NN)

Let's now discuss our results which do show a signficant difference between the random forest and the neural network:

</br>

* P-values:

* * **Random forest**: The p-value for the random forest exceeds our $5-95\%$ rule at $95.2%$. This means the KS test has determined that the test and training data responses belong to the same distribution with firm confidence. In general, a p-value like this may be a clue for us to double check that we haven't made a mistake or done anything "dodgy" to force good agreement. Here, we are fairly confident that we haven't done this.

* * **Neural network**: The p-value for the neural network falls comfortably within the $5-95\%$ rule at $42.6\%$ showing relatively strong agreement between the test and train data responses, but not quite as good as fot the random forest.

</br>

Train vs test response graphs:

* * **Random forest**: Over the majority of the response range, the normalised test and training signals visually appear to match quite well, except for the few right-most bins on the plot where the test signal response sits below the training response which is indicative of some slight overfitting.

* * **Neural network**: For responses of $0.10$ or greater, the normalised test signal frequency is mostly below the training signal frequency histogram. This is indicative of clear overfitting, which is especially present in the right-most bins.



### 5.2: Results of the Machine Learning Method ###

So far we have obtained and processed our data to be classified by two machine learning models - our neural network and our random forest. These have been trained on a subset of the Monte Carlo data while an independent subset has been used to test how well our trained classifiers work when faced with unlabelled data when compared with labelled data. At this point we have a few metrics which help us to evaluate the quality of our models.

</br>

However, now is where we begin to answer the question of how our powerful tools can be applied to the task of finding dark matter. In this section, we evaluate how well our signal is selected in and amongst background with in terms of the signal **significance**.

#### 5.2.1: Finding the significance ####

Putting all the data into a classifier means we only have one variable to optimise (the classifier response), unlike in Workbook 2 where there are many. Using the classifier output also achieves higher significance values than using individual variables because machine learning models can find complex correlations in many dimensions.

We will now find the significance of the signal by doing something called a 'scan cut' on the classifier response. If we look at the response plot and imagine a vertical line on it, we can perform a cut on the response - keeping everything to the right, and using the same significance equation as we used in Workbook 2:

$ \text{significance} = \frac{\text{signal}}{\sqrt{\text{background}}} $

Where “signal" and "background” in this equation are each the sum of all the weights of the signal or background events in the included part of the histogram (to the right of the line in the diagram below) like we did and programmed in Workbook 2.

![sigcaldia](https://lh3.googleusercontent.com/Dzh5AjymVFfv7Fs8_NF27sxKfcIukVsR8DNkn3lzS49aBRvBvBQ_2EjCw_iWll8hwK4=w2400)


We shall move this line from left to right, performing scan cutting and getting a significance value for each place we perform the cut. In turn, this can create a line plot of the significance over all of the cuts.

The code to do this can be seen below (note that NumPy masks are used to select the data from the histogram, see more on this [here]( https://www.google.com/search?client=opera-gx&q=in+unison&sourceid=opera&ie=UTF-8&oe=UTF-8)):

In [ ]:
#@title run this cell before the below one

def plot_response_alt(fig,clf,Title,sig,sig_weights,bkg,bkg_weights,n_bins,index,xAxis,Log):
    """
    This function plots the response of a classifer on signal and background 
    Monte Carlo events


    Parameters
    ----------
    fig : TYPE
        DESCRIPTION.
    clf : TYPE
        DESCRIPTION.
    Title : TYPE
        DESCRIPTION.
    sig : TYPE
        DESCRIPTION.
    sig_weights : TYPE
        DESCRIPTION.
    bkg : TYPE
        DESCRIPTION.
    bkg_weights : TYPE
        DESCRIPTION.
    n_bins : TYPE
        DESCRIPTION.
    index : TYPE
        DESCRIPTION.
    xAxis : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    
    # Add this axes to the fig at index
    ax = fig.add_subplot(index)
    
    
    # Get response (in form of probabilities) of signal and background 
    sig_prob = clf.predict_proba(sig)[:, 1]
    bkg_prob = clf.predict_proba(bkg)[:, 1]
    # find min and max for plot
    d_min = min(sig_prob.min(), bkg_prob.min())
    d_max = max(sig_prob.max(), bkg_prob.max())
    
    
    
    # plot background response (make orange and translucent)
    ax.hist(bkg_prob,
             bins = n_bins,
             range = (d_min,d_max),
             color = 'tab:blue', 
             label = 'bkg test',
             alpha = 0.4, 
             density = False,
             weights = bkg_weights,
             log = Log,
             histtype="stepfilled",
             edgecolor='tab:blue',
             linewidth=5)
    
    
    
    
    
    
    # plot signal response (make blue and translucent)
    ax.hist(sig_prob,
             bins = n_bins,
             range = (d_min,d_max),
             color = 'tab:orange', 
             label = 'sig test', 
             alpha = 0.4, 
             density = False,
             weights = sig_weights,
             log = Log,
             histtype="stepfilled",
             edgecolor='tab:orange',
             linewidth=5)
    
    
    
    
    
    
    # Set up plot
    if Title != None:
        ax.set_title(Title)
    ax.set_xlabel("Response (0 = background, 1 = signal)")
    ax.set_ylabel("Frequency")
    ax.legend()
    if not xAxis:
        ax.xaxis.set_visible(False)
    
    return

In [ ]:
# Define simpler equation to find the approximate median significance (approximation
# that background count is much larger than the signal)
def AMS_short(tpr, fpr, b_reg):
    return tpr/np.power(fpr+b_reg, 0.5)


# Plot significance from scan cutting
def plot_significance(Title, 
                      clf, 
                      X_test, 
                      X_test_weight, 
                      X_test_ID,
                      sig_test, 
                      sig_test_W,
                      bkg_test,
                      bkg_test_W):
    """
    This function calculates the significance of the signal using the response
    of the classifer

    Parameters
    ----------
    Title : TYPE
        DESCRIPTION.
    clf : TYPE
        DESCRIPTION.
    X_test : TYPE
        DESCRIPTION.
    X_test_weight : TYPE
        DESCRIPTION.
    X_test_ID : TYPE
        DESCRIPTION.
    sig_test : TYPE
        DESCRIPTION.
    sig_test_W : TYPE
        DESCRIPTION.
    bkg_test : TYPE
        DESCRIPTION.
    bkg_test_W : TYPE
        DESCRIPTION.

    Returns
    -------
    sigs : TYPE
        DESCRIPTION.

    """
    
    fig, ax = plt.subplots(2,1,sharex=True)
    #fig.tight_layout()
    [axi.set_axis_off() for axi in ax.ravel()]
    # array to hold significance vaules 
    sigs = []    
    
    # get the predictions (in form of probabilities)
    probs = clf.predict_proba(X_test)[:, 1]
    
    # make an array of prediction vaules to scan over
    prob_space = np.linspace(0,max(probs),100)
    

    #loop through prediction vaules
    for p in prob_space:
        
        # generate masks to find events that correspond to the following conditions:
        prob_mask = (probs > p)
        sig_mask = (X_test_ID == 4.0)
        bkg_mask = (X_test_ID != 4.0)
        
        # get the weights of the signal and background events that have a response
        # less than the current cutting vaule (p from prob_space)
        sig_weights = X_test_W[np.logical_and(prob_mask, sig_mask)]
        bkg_weights = X_test_W[np.logical_and(prob_mask, bkg_mask)]
        
        # Get number of signal and number of background events that have a response
        # less than the current cutting vaule (p from prob_space)
        signal_selected_num = np.count_nonzero(np.logical_and(prob_mask, sig_mask))
        background_selected_num = np.count_nonzero(np.logical_and(prob_mask, bkg_mask))
        
        # Find the sum of the weights found earlier
        signal_weights_selected = np.sum(sig_weights)
        background_weights_selected =np.sum(bkg_weights)
        
        # Calculate the significace, but if one of the weights sums to zero set sig to zero 
        if (background_weights_selected == 0.0 or signal_weights_selected == 0.0): # Avoid errors
            significance = 0
        else:
            significance = AMS_short(signal_weights_selected, background_weights_selected,0.001) 
            
        #append found significance vaule
        sigs.append(significance)
    
    
    # Add First plot
    ax = fig.add_subplot(212)
    ax.yaxis.tick_right()
    # plot significance curve from scan cutting
    ax.plot(prob_space, sigs)
    
    # Set up plot
    ax.set_xlabel("Response of classifer")
    ax.set_ylabel("Significance, $\sigma$")
     
    # We will add a second plot of the reponce histogram above the significace plot 
    plot_response_alt(fig,
                      clf, 
                      Title,               
                      sig_test, 
                      sig_test_W,
                      bkg_test,
                      bkg_test_W,         
                      BINS,
                      211,
                      False,
                      True)
    
    # Adjust spacing of the two plots on the same figure
    plt.subplots_adjust(wspace=0, hspace=0)
    fig.set_size_inches(12, 6)
    plt.show()

    return sigs

Let's call this function for the neural network:

In [ ]:
significance_NN = plot_significance("Significance from the neural network classifier", 
                                     NN_clf, 
                                     X_test_scaled, 
                                     X_test_W, 
                                     X_test_ID,
                                     sig_test, 
                                     sig_test_W,
                                     bkg_test,
                                     bkg_test_W)

And again for the random forest:

In [ ]:
significance_FR = plot_significance("Significance from the random forest classifier", 
                                     RF_clf, 
                                     X_test_scaled, 
                                     X_test_W, 
                                     X_test_ID,
                                     sig_test, 
                                     sig_test_W,
                                     bkg_test,
                                     bkg_test_W)

Let’s now find the highest significance for the signal (the dark matter signal) for each classifier:

In [ ]:
# Print max significances found for each classifer
print("NN significance: ",max(significance_NN),"    With KS test results: ",ksTest_NN)
print("RF significance: ",max(significance_FR),"    With KS test results: ",ksTest_RF)

Firstly, let's reassure ourselves with the following: we have now done all of the more involved work for developing and using neural networks. Hopefully everything South of this paragraph will be reusing ideas which we have already discussed in this workbook. 

</br>

Secondly, let's now take a moment to round off our evaluation of our two models:

* **Neural network:** Despite our previous comments regarding the results of our KS-test and our comparison of the test and training signal responses, we should note that the neural network looks like it can be used to obtain a sample with a DM signal significance of $2.37\sigma$. Could $3\sigma$ actually be reached while keeping our pesky overfitting to a minimum?

* **Random forest:** Although we previously noted some reassuring results from our overfitting and response tests, the signal significance from this model is slightly lower than the neural network at $2.26\sigma$. Given that random forests are generally "simpler" than neural networks, these results are not surprising (if you are surprised, feel free to look back throughout this workbook). 

#### 5.2.2: Train your own classifier ####
Now it is time to train your own classifier using everything you have learnt, you shouldn't have to do anything which you haven't seen before in this notebook so use this as a chance to consolidate your knowledge so far. Feel free to look back through the notebook if you are unsure on what to do.

#### Step 1: Pre processing ####

Using the premade X and y datasets perform the train test split and clean up the results (remove IDs and weights for later use, see section 3.1)
Initialise the classifier with the machine learning method of your choice (Neural Network, Random Forest or [other](https://www.google.com)) using your starting hyperparameters.

In [ ]:
#Write your code in in this cell 

#### Step 2: Train the classifier ####

Now train your classifer, this is a simple step where you fit the 

In [ ]:
#Write your code in in this cell 

#### Step 3: Analyse the classifier ####


Analyse your classifier using the prewritten functions.

In [ ]:
#Write your code in in this cell 

##### plot_response() #####

**clf** → classifier object that you trained
 
**Title** → title of your plot
 
**sig, sig_weights** → numpy arrays for signal events and corresponding weights
 
**bkg, bkg_weights** → numpy arrays for background  events and corresponding weights
 
**n_bins** → number of bins to plot histogram with
 
**Log** → boolean to get y axis to be logarithmic or not 


##### calc_roc() #####

**clf** → classifier object that you trained
 
**X, X_weight** → premade x dataset and corresponding weights
 
**y** →  premade y dataset
 
**Title** → title of your plot

##### plot_overfittingCheck() #####

 
**clf** → classifier object that you trained
 
**Title** → title of your plot                    
 
**sig_train, sig_train_weights, sig_test, sig_test_weights** →  numpy arrays for 
train/test signal events and corresponding weights
 
**bkg_train, bkg_train_weights, bkg_test, bkg_test_weights** → numpy arrays for train/test background events and corresponding weights
 
**n_bins** → number of bins to plot histogram with

#### Step 4: Retrain ####

Retrain your classifier if it does not meet your standards then re-analyse your classifier.

In [ ]:
#Write your code in in this cell 

#### Step 5: Calculate the significance ####

Now calculate the significance of the signal using the prewritten function

In [ ]:
#Write your code in in this cell 

##### plot_significance() #####

**Title** → title of your plot
 
**clf** → classifier object that you trained
 
**X_test, X_test_weight, X_test_ID** → x test dataset and corresponding weights and IDs
 
**sig_test, sig_test_W** →   numpy arrays for test signal events and corresponding weights
 
**bkg_test, bkg_test_W** →  numpy arrays for test background events and corresponding weights


### Section 5 quiz ###

In [ ]:
#@title
print("Often when developing machine learning models, there is a trade-off between maximising the \n"+
      "TPR and minimising the FPR. If you are attempting to find a really clear signal, but are \n"+
      "confident that you have enough data to discard if need be, which of the two are you most concerned with?")
out = widgets.Dropdown(options=[('',0),('Maximising TPR',1),
                                ('Minimising FPR',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print(f"In principle, it is possible to perform goodness of fit tests using several different methods, \n"+
      "dependent upon the situation. One example of a test which is often used in physics is the \nchi-squared"+
      " test. Let\'s say we wish to compare the fits in the results from two different analyses -\n"+
      "one using "+
      "the chi-squared and another using the KS-test. Which metric do we want to use?")
out = widgets.Dropdown(options=[('',0),('Significance',0),
                                ('KS statistic',0), ("The p-value",1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In one analysis (A), results are found to match a null hypothesis with a p-value of 90%. \n"+
      "In another analysis (B), results match the same hypothesis with a p-value of 10%. \n"+
      "Which experiment shows stronger agreement with the null hypothesis?")
out = widgets.Dropdown(options=[('',0),('A',1),
                                ('B',0)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In the above analysis which we did, which machine learning model demonstrated stronger evidence of overfitting?")
out = widgets.Dropdown(options=[('',0),('Random forest',0),
                                ('Neural network',1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
#@title
print("In the same analysis, which machine learning model demonstrated evidence of finding a greater signal significance?")
out = widgets.Dropdown(options=[('',0),('Random forest',0),
                                ('Neural network',1)],description='Answer:',disabled=False)
def drop_check(guess):
  if guess==1:
    print("\033[1;32;47m Correct!  \n")
  else:
    print("\033[1;32;47m Incorrect.  \n")
check = widgets.interactive_output(drop_check,{'guess':out})
widgets.HBox([out,check])

In [ ]:
data_inputs = ['lead_lep_pt', 
             'sublead_lep_pt', 
             'mll',
             'ETmiss', 
             'dphi_pTll_ETmiss', 
             'fractional_pT_difference',
             'ETmiss_over_HT']

data = pd.read_csv("/content/drive/MyDrive/Data/data.csv")
data = data[data_inputs].copy()
data = np.asarray(data)
scaled_data = scaler.transform(data)

##This cell imports all of the real data that we need in the same format as in previous MC-driven sections 

There are however a few differences which actually make our job quite a bit easier. Firstly we only have one file rather than several processes. This means that there is no need to combine several dataframes together. Secondly, since the data is real, it is entirely nonsensical to think about sample weights or reweighting, so we don't account for it when importing the data. Finally, we do not have to split data up between test and train datasets since all model evaluation has already been completed. 

However, we do still need to make sure that the data is imported with exactly the same input variables, and in a consistent manner to when the MC data were imported. Namely, we had to make sure that our numpy array for the data input, event-by-event, had the same ordering as with the MC data input. We also still have to scale our data input, since the ML models were trained and tested with normalised data inputs to help them to work.  

Please take the time now to convince yourself that things are consistent between this importing of the real data and the way MC data were used in previous sections.

Apply the NN and RF that you trained in Section 5.2.2 to the real data to predict if the events are signal or noise. In a similar fashion as in Section 5.1.3, we can compare the distribution of ML output when applied to the data against the distribution of ML output when applied to the test MC dataset. 

This allows us to assess how consistent the ML models are behaving to when they were trained and tested. Hopefully the importance of this has already been made clear to you by now; **any inconsistency now will either indicate the existence of dark matter or indicate potential improvements to our analysis to enable us to do that.** 

The code hidden below defines a function to plot the data's ML response against the test MC sample's (with the test sample reweighted to the larger, full dataset's size). A KS test is also performed which assesses how well the data response distribution matches the response due to the MC test **signal** sample *only*. We do this because we want to try to prove the existence of dark matter which isn't known within our current scientific understanding. Therefore, we have to consider the presence of DM as an alternate hypothesis $H_1$ against a null hypothesis $H_0$ that take as *only* background due to known Standard Model processes.

Please refere back to Section 5.1.3 if you need a refresheron this before proceeding.

In [ ]:
#@title run this hidden code to continue
def plot_data(clf, Title,                        
                          data, sig_test, sig_test_weights,
                          bkg_test, bkg_test_weights,       
                          n_bins, test_size=0.33, cut=None):
    """
    This funtion plots the ML responses for both real data and the rescaled test dataset. 
    This rescaling ensures that the total number of events roughly matches in both cases.
    It will also display the results
    of a Kolmogorov–Smirnov test comparing the test data signal response and real data response.
    This acts as a hypothesis test where H0 is SM-background-only.

    Will output data, test signal and test background responses binned according to
    the output bin centres.

    A cut can also be applied as a list or 1-D array as [lower_lim, upper_lim]
    where lower_lim and upper_lim give the intended selection cuts.

    """
    
    #net_W = sum(sig_test_weights)+sum(bkg_test_weights)
    
    # get the predictions (in form of probabilities) for background and 
    # signal for the train data set and the test data set
    data_prob = clf.predict_proba(data)[:, 1]
    #bkg_train_prob = clf.predict_proba(bkg_train)[:, 1]
    
    sig_test_prob = clf.predict_proba(sig_test)[:, 1]
    bkg_test_prob = clf.predict_proba(bkg_test)[:, 1]
    
    #Applying cuts
    if cut:
      data_prob = data_prob[data_prob>cut[0]]
      data_prob = data_prob[data_prob<cut[1]]

      sig_test_weights = sig_test_weights[np.logical_and(sig_test_prob>cut[0], sig_test_prob<cut[1])]
      #sig_test_weights = sig_test_weights[sig_test_prob<cut[1]]
      sig_test_prob = sig_test_prob[np.logical_and(sig_test_prob>cut[0], sig_test_prob<cut[1])]
      #sig_test_prob = sig_test_prob[sig_test_prob<cut[1]]   

      bkg_test_weights = bkg_test_weights[np.logical_and(bkg_test_prob>cut[0], bkg_test_prob<cut[1])]
      #bkg_test_weights = bkg_test_weights[bkg_test_prob<cut[1]]
      bkg_test_prob = bkg_test_prob[np.logical_and(bkg_test_prob>cut[0], bkg_test_prob<cut[1])]
      #bkg_test_prob = bkg_test_prob[bkg_test_prob<cut[1]]      

    # find min and max for plot
    d_min = min(sig_test_prob.min(), bkg_test_prob.min(), data_prob.min())
    d_max = max(sig_test_prob.max(), bkg_test_prob.max(), data_prob.max())
    
    
    
    
    # plot the background response of the test data set (make orange and translucent)
    bkg_te, bins, _ = plt.hist(bkg_test_prob,
                               bins=n_bins,
                               range=(d_min,d_max), 
                               color='tab:blue', 
                               label='bkg test',
                               alpha=0.4,
                               weights = bkg_test_weights/test_size,
                               log = True,
                               histtype="stepfilled",
                               edgecolor='tab:blue',
                               linewidth=5)
    
    # plot the signal response of the test data set (make blue and translucent)
    sig_te, bins, _ = plt.hist(sig_test_prob,
                              bins=n_bins,
                              range=(d_min,d_max), 
                              color='tab:orange', 
                              label='sig test', 
                              alpha=0.4,
                              weights = sig_test_weights/test_size,
                              bottom=bkg_te,
                              stacked=True,
                              log = True,
                              histtype="stepfilled",
                              edgecolor='tab:orange',
                              linewidth=5)
    # plots shaded bars for an assumed error on the MC data of 10%
    MCerror = 0.1*(sig_te+bkg_te)
    plt.bar(bins[:-1], MCerror, width=bins[1]-bins[0], bottom=sig_te+bkg_te, align="edge",
            log=True,  color="tab:gray", edgecolor="tab:gray", linewidth=5, label="MC error (10%)")
    
    

    # find bin centres
    bin_centers = (bins[:-1] + bins[1:])/2
    # use numpy histograms to find heights of histogram bars for the real data set
    data_hist, _ = np.histogram(data_prob, bins = bins)
    #bkg_te, _ = np.histogram(bkg_test_prob, bins = bins, weights = bkg_test_weights/test_size)
    
    # plot the real data as points with an error bar, assumed to be equal to a standard Poisson uncertainty of sqrt(n) 
    plt.errorbar(bin_centers,data_hist, yerr=np.sqrt(data_hist), fmt='o', 
                 capsize=3, c='tab:blue', label = 'data', alpha = 0.9, markeredgecolor = 'k')
    #plt.plot(bin_centers,sig_te, 'o', c = 'tab:orange', label = 'sig test', alpha = 0.9, markeredgecolor = 'k')
    
    # set up the plot
    plt.title(Title)
    plt.xlabel("Response (0 = background, 1 = signal)")
    plt.ylabel("Normalised frequency")
    plt.yscale("log")
    plt.legend()
    
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    
    plt.show()
    
    tot_test = bkg_te+sig_te

    # return the Kolmogorov–Smirnov test results
    #print(sum(data_hist), sum(tot_test))
    print(ks_2samp(data_hist, bkg_te))
    return data_hist, sig_te, bkg_te, bin_centers

Let's plot the response for the random forest:

In [ ]:
RF_data, RF_sig, RF_bkg, RF_bin_centers = plot_data(RF_clf, "RF", scaled_data, sig_test, 
                                     sig_test_W,
                                     bkg_test,
                                     bkg_test_W, 10, 0.33)

Now for the neural network:

In [ ]:
NN_data, NN_sig, NN_bkg, NN_bin_centers =  plot_data(NN_clf, "NN", scaled_data, sig_test, 
                                     sig_test_W,
                                     bkg_test,
                                     bkg_test_W, 10, 0.33)

Brilliant! We have started to assess results now with **actual data**. I know, it is pretty amazing. Let's discuss the distributions and the p-values before we move on:

P-values:

* Random forest: When we consider the full range of RF classifier responses, the KS test gives us a **$99.4\%$** chance that the data is identical to the background-only test response.

* Neural network: Over a full range of NN responses, the KS test returns a **$78.7\%$** probability that the data is the same as a background-only response. 

OK, so the obtained p-values are not looking particularly reassuring in trying to prove our $H_1$ DM hypothesis. There are two things which are worth pointing out, however, which do allow room for hope. 

Firstly, the KS tests in this workbook are limited by the fact that they allow the two samples in the tests to be different sizes - only the relative trends make a difference. Yet, let's look at the difference in data and test sample plots:

* Random forest: Despite the test MC sample being renormalised to the same size as the data, there is shown an excess of data events above test sample's signal.

* Neural network: We see a similar pattern of behaviour for the NN too. In fact, one may ask if the data excess could be explained away by the the existence of a DM signal, which the test MC plots show may contribute to this excess.

Secondly, the ML models were developed as a tool to separate DM signal and SM background, to allow for a suitable enough significance by which to confirm dark matter. 

In the following sections, we will build on these remarks, first by applying cuts on the ML models to optimise the significance, then by considering a slightly more involved method of hypothesis testing in high-energy physics.

## 6: conclusion ##
This workbook describes how to use machine learning to find dark matter within the ATLAS open  data project, working on supervised learning. This is when we train a machine learning model with example data of what we are looking for then the program will perform a task(find dark matter production process).
<br>
<br>
We would also appreciate if you could fill out our evalutation form and quiz yourself using the link [here!](https://docs.google.com/forms/d/e/1FAIpQLSfXG7-MMOamaIhnWJT_UVmUbNjZ_INuyWto353HuNPiJQvAMQ/viewform?usp=sf_link)

## 7: Credits and licencing  ##

Project Lead: 		Kate Shaw
<br>
<br>
Project Supervisors: 	Meirin Oan Evans, Zöe Earnshaw, Thomas Stevenson
<br>
<br>
Developers: 		Christopher Comiskey-Erazo, Iago Rosetto, Oscar Jackson